In [85]:
from pathlib import Path

import pandas as pd
from promg.modules.db_management import DBManagement
from promg import Configuration, DatabaseConnection, Performance, SemanticHeader, DatasetDescriptions, OcedPg, Query

import yaml

import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

In [2]:
case_study = 'bpic14'

conf_path = Path(case_study, 'config.yaml')
config = yaml.safe_load(open(conf_path))

print(f"These are the credentials that I expect to be set for the database.")
print(f"db_name: {config['db_name']}")
print(f"uri: {config['uri']}")
print(f"password: {config['password']}")
print("----------------------")
print(f"If you have other credentials, please change them at: {conf_path}")

config = Configuration.init_conf_with_config_file(conf_path)
db_connection = DatabaseConnection.set_up_connection(config=config)

These are the credentials that I expect to be set for the database.
db_name: neo4j
uri: bolt://localhost:7687
password: bpic2014
----------------------
If you have other credentials, please change them at: bpic14\config.yaml


In [ ]:
# clear previously created nodes and relations
# start/end relations
qClearStart = f'''MATCH (n)<-[r:START]-(e:Event) DELETE r'''
db_connection.exec_query(qClearStart)
qClearEnd = f'''MATCH (n)<-[r:END]-(e:Event) DELETE r'''
db_connection.exec_query(qClearEnd)
# high-level events
qClearStartHLE = f'''MATCH (n)<-[r:START]-(e:HighLevelEvent) DELETE r'''
db_connection.exec_query(qClearStartHLE)
qClearEndHLE = f'''MATCH (n)<-[r:END]-(e:HighLevelEvent) DELETE r'''
db_connection.exec_query(qClearEndHLE)
qClearHighLevelEvent = f'''MATCH (h:HighLevelEvent) DETACH DELETE h'''
db_connection.exec_query(qClearHighLevelEvent)
qClearHL_DF = f'''MATCH (:Event)-[r:HL_DF]-(e:Event) DELETE r'''
db_connection.exec_query(qClearHL_DF)

In [11]:
qClearStart = '''
:auto
MATCH (n)<-[r:START_CORR]-(e:HighLevelEvent)
CALL (r) {
    DELETE r
} IN TRANSACTIONS
'''

db_connection.exec_query(qClearStart)
qClearEnd = '''
:auto
MATCH (n)<-[r:END_CORR]-(e:HighLevelEvent)
CALL (r) {
    DELETE r
} IN TRANSACTIONS
'''
db_connection.exec_query(qClearEnd)

[]

In [31]:
qClearDuring = '''
:auto
MATCH (:HighLevelEvent)<-[r:DURING]-(:HighLevelEvent)
CALL (r) {
    DELETE r
} IN TRANSACTIONS
'''
db_connection.exec_query(qClearDuring)

[]

In [10]:
# fix DF relations for incidents
qDeleteDFIncidents = f'''
    MATCH (e1:Event)-[df:DF {{objectType:"Incident"}}]->(e2:Event)
    DELETE df
'''
db_connection.exec_query(qDeleteDFIncidents)

with db_connection.driver.get_session(database=db_connection.db_name) as session:
    qBuildDFIncidents = f'''
        MATCH (e:Event)-[:CORR]->(o:Incident)
        WHERE e.timestamp IS NOT NULL
        WITH o, e ORDER BY e.timestamp, ID(e)
        WITH o, collect(e) AS events
        UNWIND range(0, size(events)-2) AS idx
        WITH events[idx] AS fromEv, events[idx+1] AS toEv, o.sysId AS objSysId
        MERGE (fromEv)-[rel:DF {{objectType:"Incident", id:objSysId}}]->(toEv)
    '''
    result = session.run(qBuildDFIncidents)
    result.consume()
    print(str(result.single))

<bound method Result.single of <neo4j._sync.work.result.Result object at 0x00000221C9219910>>


In [5]:
objectTypes = ["Change", "Incident", "Interaction"]

In [4]:
# infer start and end events for each object type

for objectType in objectTypes:
    print(f"Inferring start and end events for object type: {objectType}")

    qStartEvent = f'''
    // Infer start event of an object
    MATCH (n:{objectType})
    MATCH (n)<-[c:CORR]-(e:Event) WHERE NOT ()-[:DF {{id:n.sysId}}]->(e)
    MERGE (n)<-[:START]-(e)
    '''
    result = db_connection.exec_query(qStartEvent)
    qEndEvent = f'''
    // Infer start event of an object
    MATCH (n:{objectType})
    MATCH (n)<-[c:CORR]-(e:Event) WHERE NOT (e)-[:DF {{id:n.sysId}}]->()
    MERGE (n)<-[:END]-(e)
    '''
    db_connection.exec_query(qEndEvent)

Inferring start and end events for object type: Change
Inferring start and end events for object type: Incident
Inferring start and end events for object type: Interaction


In [5]:
# materialize time between events as duration on DF edge
with db_connection.driver.get_session(database=db_connection.db_name) as session:
    q_materialize_duration_on_df = f'''
        MATCH (e1:Event)-[df:DF]->(e2:Event) WHERE df.duration IS NOT NULL
        CALL {{
            WITH e1,e2,df
            SET df.duration = duration.inSeconds(e1.timestamp, e2.timestamp).seconds    
        }} IN TRANSACTIONS OF 1000 ROWS
        RETURN count(*) AS total
    '''
    result = session.run(q_materialize_duration_on_df)
    result.consume()
    print(str(result.single))

Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery without a variable scope clause is deprecated. Use CALL (e1, e2, df) { ... }', position=<SummaryInputPosition line=3, column=9, offset=84>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 84, 'line': 3, 'column': 9}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n        MATCH (e1:Event)-[df:DF]->(e2:Event) WHERE df.duration IS NOT NULL\n        CALL {\n            WITH e1,e2,df\n            SET df.duration = duration.inSeconds(e1.timestamp, e2.timestamp).seconds    \n        } IN TRANSACTIONS OF 1000 ROWS\n        RETURN count(*) AS total\n    '


<bound method Result.single of <neo4j._sync.work.result.Result object at 0x000001A638B528D0>>


In [7]:
# build index for high-level events
qCreateIndexHLE_sys = f'''
    CREATE INDEX highLevelEventSysIdIndex IF NOT EXISTS FOR (h:HighLevelEvent) ON (h.sysId);
'''
db_connection.exec_query(qCreateIndexHLE_sys)
qCreateIndexHLE_start = f'''
    CREATE INDEX highLevelEventStartTimeIndex IF NOT EXISTS FOR (h:HighLevelEvent) ON (h.startTime);
'''
db_connection.exec_query(qCreateIndexHLE_start)
qCreateIndexHLE_end = f'''
    CREATE INDEX highLevelEventEndTimeIndex IF NOT EXISTS FOR (h:HighLevelEvent) ON (h.endTime);
'''
db_connection.exec_query(qCreateIndexHLE_end)

[]

In [13]:
# build high-level events
qBuildHighLevelEvent = '''
    :auto
    MATCH (n) - [:IS_OF_TYPE] -> (ot:ObjectType {objectType: $objectType})
    MATCH (eStart:Event)-[st:START]->(n)<-[en:END]-(eEnd:Event)
    WITH DISTINCT eStart, eEnd
    CALL (eStart, eEnd) {
        MERGE (h:HighLevelEvent {sysId: eStart.sysId + "_" + eEnd.sysId})
        ON CREATE SET h.startTime=eStart.timestamp, h.endTime=eEnd.timestamp, h.activity=$objectType
        MERGE (h)-[:START]->(eStart)
        MERGE (h)-[:END]->(eEnd)
    } IN TRANSACTIONS
'''

for objectType in objectTypes:
    query = Query(
        query_str=qBuildHighLevelEvent,
        parameters={
            "objectType": objectType,
        }
    )

    db_connection.exec_query(query)

In [15]:
# lift CORR to high-level events
qLiftCorrToHL = '''
    :auto
    MATCH (h:HighLevelEvent)-[:START]->(eStart:Event)-[:CORR]->(n)
    CALL (h, n) {
        MERGE(h) - [:CORR]->(n)
    } IN TRANSACTIONS
'''
db_connection.exec_query(qLiftCorrToHL)

[]

In [3]:
# build DF relations between high-level events
# step 1: build HL-DF relations between events

qBuildDfRelationsHLE = '''
    :auto
    // get only START/END events of each HL event correlated to CI_SC
    MATCH (ot:ObjectType {objectType: $objectType}) <- [:IS_OF_TYPE] - (o) <-[:CORR]-(h:HighLevelEvent)-->(e:Event)
    // order by time
    WHERE e.timestamp IS NOT NULL
    WITH o, e ORDER BY e.timestamp, ID(e)
    WITH o, collect(e) AS events
    UNWIND range(0, size(events)-2) AS idx
    WITH events[idx] AS fromEv, events[idx+1] AS toEv, o
    CALL (fromEv,toEv,o) {
        // get their HL events correlated to the same object
        MATCH (fromEv)<--(h1:HighLevelEvent)-[:CORR]->(o)<-[:CORR]-(h2:HighLevelEvent)-->(toEv) WHERE h1 <> h2
        // and add their DF relation
        MERGE (h1)-[rel:DF {objectType:$objectType, id:o.sysId}]->(h2)
    } IN TRANSACTIONS
    RETURN count(*) AS total
'''

In [6]:
for objectType in objectTypes:
    query = Query(
        query_str=qBuildDfRelationsHLE,
        parameters={
            "objectType": objectType,
        }
    )

    db_connection.exec_query(query)

In [18]:
# infer start and end events for each object type

object_type = "CI_SC"
print(f"Inferring start and end events for object type: {object_type}")

q_start_event_str = '''
:auto
// Infer start event of an object
MATCH (ot:ObjectType {objectType: $objectType}) <- [:IS_OF_TYPE] - (n)
CALL (n) {
    MATCH (n) <- [:CORR] - (e:HighLevelEvent)
    RETURN e ORDER BY e.startTime, id(e) LIMIT 1
}
CALL (n, e) {
    MERGE (n)<-[:START_CORR]-(e)
} IN TRANSACTIONS
RETURN count(n)
'''

q_start_event = Query(
    query_str=q_start_event_str,
    parameters={
        "objectType": object_type,
    }
)

db_connection.exec_query(q_start_event)

Inferring start and end events for object type: CI_SC


[{'count(n)': 22259}]

In [24]:


q_end_event_str = '''
:auto
// Infer end event of an object
MATCH (ot:ObjectType {objectType: $objectType}) <- [:IS_OF_TYPE] - (n)
CALL (n) {
    MATCH (n) <- [:CORR] - (e:HighLevelEvent)
    RETURN e ORDER BY e.endTime DESC, id(e) LIMIT 1
}
CALL (n, e) {
    MERGE (n)<-[:END_CORR]-(e)
} IN TRANSACTIONS

RETURN count(n)
'''

q_end_event = Query(
    query_str=q_end_event_str,
    parameters={
        "objectType": object_type,
    }
)

db_connection.exec_query(q_end_event)

[{'count(n)': 22259}]

In [3]:
# find overlapping high-level events and create DURING relations
qFindContainedHLEvents = '''
:auto
    MATCH (h1:HighLevelEvent) - [df:DF] -> (h2:HighLevelEvent) - [:DF {id: df.id}] -> (h1)
    WHERE h1 <> h2 AND h1.startTime < h2.startTime AND h2.endTime < h1.endTime
      CALL (h1, h2) {
        MERGE (h2)-[:DURING]->(h1)
    } IN TRANSACTIONS
    RETURN count(h2) AS total
'''
db_connection.exec_query(qFindContainedHLEvents)

[{'total': 18823}]

In [ ]:
# find overlapping high-level events and create DURING relations

qFindContainedHLEvents = '''
:auto
    MATCH (h1:HighLevelEvent) - [df:DF] -> (h2:HighLevelEvent) - [df2:DF {id: df.id}] -> (h1)
    WHERE h1 <> h2 AND h1.startTime < h2.startTime AND h2.endTime < h1.endTime
      CALL (h1, h2) {
        MERGE (h2)-[:DURING]->(h1)
    } IN TRANSACTIONS
    RETURN count(h2) AS total
'''
db_connection.exec_query(qFindContainedHLEvents)

In [ ]:
# lift DF relations from events to high-level events
qLiftDfRelations = '''
    :auto
    MATCH (h1:HighLevelEvent)-[:END]->(e1:Event)-[df:DF]->(e2:Event)<-[:START]-(h2:HighLevelEvent)
    CALL (h1, df, h2) {
        MERGE (h1)-[df2:DF]->(h2)
        ON CREATE SET df2 = properties(df)
    }
    YIELD total
    RETURN total
'''
result = db_connection.exec_query(qLiftDfRelations)
print(result)



In [ ]:
# lift DF relations from events to high-level events
with db_connection.driver.get_session(database=db_connection.db_name) as session:
    qLiftDfRelations = f'''
        MATCH (h1:HighLevelEvent)-[:END]->(e1:Event)-[df:DF]->(e2:Event)<-[:START]-(h2:HighLevelEvent)
        CALL {{
            WITH h1,df,h2
            MERGE (h1)-[df2:DF]->(h2) ON CREATE SET df2 = properties(df)
        }} IN TRANSACTIONS OF 1000 ROWS
        RETURN count(*) AS total
    '''
    result = session.run(qLiftDfRelations)
    result.consume()
    print(result.single)

In [ ]:
# find overlapping high-level events and create DURING relations
with db_connection.driver.get_session(database=db_connection.db_name) as session:
    qFindContainedHLEvents = f'''
        MATCH (h1:HighLevelEvent)-[c1:CORR]->(n:CI_SC)<-[c2:CORR]-(h2:HighLevelEvent) WHERE h1 <> h2
        WHERE h1.startTime < h2.startTime AND h2.endTime < h1.endTime
        CALL {{
            WITH h1,h2
            MERGE (h2)-[:DURING]->(h1)
        }} IN TRANSACTIONS OF 1000 ROWS
        RETURN count(*) AS total
    '''
    result = session.run(qFindContainedHLEvents)
    result.consume()
    print(result.single)